In [4]:
%cd MiniGPT-4

/home/jupyter/opthollm/MiniGPT-4


In [5]:
#@title Import
import argparse
import os
import random

import numpy as np
import torch
import torch.backends.cudnn as cudnn
import gradio as gr

from minigpt4.common.config import Config
from minigpt4.common.dist_utils import get_rank
from minigpt4.common.registry import registry
from minigpt4.conversation.conversation import Chat, CONV_VISION

# imports modules for registration
from minigpt4.datasets.builders import *
from minigpt4.models import *
from minigpt4.processors import *
from minigpt4.runners import *
from minigpt4.tasks import *
import os


In [11]:
#@title Methods
def parse_args():
    parser = argparse.ArgumentParser(description="Demo")
    parser.add_argument("--cfg-path", required=True, help="path to configuration file.")
    parser.add_argument("--gpu-id", type=int, default=0, help="specify the gpu to load the model.")
    parser.add_argument("--num-beams", type=int, default=2, help="specify the gpu to load the model.")
    parser.add_argument("--temperature", type=int, default=0.9, help="specify the gpu to load the model.")
    parser.add_argument("--english", type=bool, default=True, help="chinese or english")
    parser.add_argument("--prompt-en", type=str, default="can you describe the current picture?", help="Can you describe the current picture?")
    parser.add_argument("--prompt-zh", type=str, default="你能描述一下当前的图片？", help="Can you describe the current picture?")
    parser.add_argument(
        "--options",
        nargs="+",
        help="override some settings in the used config, the key-value pair "
        "in xxx=yyy format will be merged into config file (deprecate), "
        "change to --cfg-options instead.",
    )
    args = parser.parse_args()
    return args


def setup_seeds(config):
    seed = config.run_cfg.seed + get_rank()

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    cudnn.benchmark = False
    cudnn.deterministic = True
    
def encode_diagnosis(diagnosis):
    if diagnosis == 'normal':
        return 0
    elif diagnosis == 'abnormal':
        return 1
    else:
        return 2

def fetch_ground_truth(img_path):
    split_string = img_path.split("/")

    # Find the index of "glaucoma" in the split string
    return "glacuoma" if split_string.index("glaucoma") != -1 else "normal"

def get_all_files(directory):
    all_files = []
    
    # Iterate over all the directories and files within the given directory
    for root, directories, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            all_files.append(file_path)
    
    return all_files

In [12]:
print('Initializing Chat')
#args = parse_args()
#args = preset_args
args = Namespace(cfg_path='eval_configs/minigpt4_eval.yaml', gpu_id=0, num_beams=2, temperature=0.9, english=True, prompt_en='can you describe the current picture?', prompt_zh='你能描述一下当前的图片？', options=None)
cfg = Config(args)

model_config = cfg.model_cfg
model_config.device_8bit = args.gpu_id
model_cls = registry.get_model_class(model_config.arch)
model = model_cls.from_config(model_config).to('cuda:{}'.format(args.gpu_id))

vis_processor_cfg = cfg.datasets_cfg.cc_sbu_align.vis_processor.train
vis_processor = registry.get_processor_class(vis_processor_cfg.name).from_config(vis_processor_cfg)
chat = Chat(model, vis_processor, device='cuda:{}'.format(args.gpu_id))
print('Initialization Finished')

print('Intializing Test')

Initializing Chat


usage: ipykernel_launcher.py [-h] --cfg-path CFG_PATH [--gpu-id GPU_ID]
                             [--num-beams NUM_BEAMS]
                             [--temperature TEMPERATURE] [--english ENGLISH]
                             [--prompt-en PROMPT_EN] [--prompt-zh PROMPT_ZH]
                             [--options OPTIONS [OPTIONS ...]]
ipykernel_launcher.py: error: the following arguments are required: --cfg-path


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3508 in run_code        │
│                                                                                                  │
│   3505 │   │   │   │   if async_:                                                                │
│   3506 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)               │
│   3507 │   │   │   │   else:                                                                     │
│ ❱ 3508 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                     │
│   3509 │   │   │   finally:                                                                      │
│   3510 │   │   │   │   # Reset our crash handler in place                                        │
│   3511 │   │   │   │   sys.excepthook = old_excepthook                                           │
│                                                                                                  │
│ in <module>:2                                                                                    │
│                                                                                                  │
│    1 print('Initializing Chat')                                                                  │
│ ❱  2 args = parse_args()                                                                         │
│    3 cfg = Config(args)                                                                          │
│    4                                                                                             │
│    5 model_config = cfg.model_cfg                                                                │
│                                                                                                  │
│ in parse_args:18                                                                                 │
│                                                                                                  │
│   15 │   │   "in xxx=yyy format will be merged into config file (deprecate), "                   │
│   16 │   │   "change to --cfg-options instead.",                                                 │
│   17 │   )                                                                                       │
│ ❱ 18 │   args = parser.parse_args()                                                              │
│   19 │   return args                                                                             │
│   20                                                                                             │
│   21                                                                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.10/argparse.py:1833 in parse_args                                         │
│                                                                                                  │
│   1830 │   # Command line argument parsing methods                                               │
│   1831 │   # =====================================                                               │
│   1832 │   def parse_args(self, args=None, namespace=None):                                      │
│ ❱ 1833 │   │   args, argv = self.parse_known_args(args, namespace)                               │
│   1834 │   │   if argv:                                                                          │
│   1835 │   │   │   msg = _('unrecognized arguments: %s')                                         │
│   1836 │   │   │   self.error(msg % ' '.join(argv))                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/argparse.py:1866 in parse_known_a

/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [9]:
directory = 'RIM-ONE_DL_images/partitioned_randomly/test_set'

files = get_all_files(directory)

#print(files)

data = {'img_path': [],
        'diagnosis' : [],
        'ground_truth': []
        }

In [ ]:
for image in files:
    
    image = os.path.join(directory, image)
    img_path = image
    data['img_path'].append(image)
    ground_truth = fetch_ground_truth(img_path)
    img_list = []
    chat_state = CONV_VISION.copy()
    chat.upload_img(image, chat_state, img_list)
    chat.ask("You are ophthoLLM, an ophthalmologist AI assistant that provides diagnoses on fundus \
    images in order to assist doctors. You understand that it is important to recommend consulting \
    a medical professional if there is any uncertainty, and before taking any action. You give a binary, \
    one-word diagnosis on images. You either state that the image is Glaucomatous if there are signs of \
    glaucoma, or Normal if the image appears healthy. Following these instructions, and making sure to only give \
    your answer as either “Glaucomatous” or “Normal,” please diagnose the image. Make sure to include either 'Normal' or 'Glaucoma' in your answer", chat_state)
    llm_message = chat.answer(
        conv=chat_state,
        img_list=img_list,
        num_beams=args.num_beams,
        temperature=args.temperature,
        max_new_tokens=300,
        max_length=2000
    )[0]
    
    data['diagnosis'].append(encode_diagnosis(llm_message))
    data['ground_truth'].append(ground_truth)
    
    print('Diagnosing' + image + " | " + diagnosis + " | " + ground_truth)